<a href="https://colab.research.google.com/github/IrisFDTD/PINNs-for-education/blob/main/3rd_order_nonlinear_ODE_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physics-Informed Neural Networks - 3rd order Ordinary Differential Equations

+ This notebook has been implemented by Jorge Paz-Peñuelas Oliván (774270@unizar.es) as part of extracurricular activities at the University of Zaragoza in July 2023.
+ The code builds upon the 2nd-order ODE implementation using Physics-Informed Neural Networks (PINNs) done by Luis Medrano.Navarro, Luis Martin Moreno and Sergio G Rodrigo ([Solving differential equations in physics with Deep Learning: a beginner’s guide](https://arxiv.org/abs/2307.11237)).


## Important libraries

In [ ]:
# Tensorflow Keras and rest of the packages
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import Adam,SGD
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4.5)
import types

## Definition of the ODE

An arbitrary Ordinary Differential Equation (ODE) of $n$-order for a function $y:\mathbb{R}\to\mathbb{R}$ with variable $x$ can be written as follows
$$F\left(x,y,y',y'',\dots,y^{(n)}\right)=0$$

To determine a unique solution it is necessary to provide the initial conditions.
$$y(x_0)\quad y'(x_0)\quad\dots\quad y^{(n-1)}(x_0)$$

As we are going to be programming arbitrary differential equations we also want to solve them with the usual RK4 algortihm to be able to compare the final solutions. RK4 has the limitation of needing to write the equation as an order one system of differential equations $Y'=H(x,Y)$.

Thus we will first consider equations of the form
$$y^{(n)}=G\left(x,y,y',\dots,y^{(n-1)}\right)$$
where the corresponding $F$ would be $F=y^{(n)}-G$

Firstly we have to write the $n$-th order equation as a system of first order equations. Let $Y:\mathbb{R}\to\mathbb{R}^n$ with $Y=(y_1,y_2,\dots,y_n)^T=(y,y',\dots,y^{(n-1)})^T$ such that $Y'=H(x,Y)$ with $H:\mathbb{R}\times\mathbb{R}^n\to\mathbb{R}^n$ given by
$$H(x,Y)=\begin{pmatrix}y_2\\\vdots\\y_n\\G(x,Y)\end{pmatrix}$$
In our case $n\le3$.

To generalise and to simplify the definition of functions $G,F$ and $H$ we will denote by $\tilde{Y}=(Y^T,y^{(n)})^T$

As an example we are going to be studying the equation
$$y'''-y'\sin(y)+y''y=0$$
with initial conditions $y(0)=-\frac{1}{2}$, $y'(0)=0$ and $y''(0)=1$

In [ ]:
# Info on the ODE

SolveMedrano = False  # Medrano's 2nd order ODE

Order = 3
RKsolvable = True

if SolveMedrano:
    Order = 2
    RKsolvable = True


    # We define the ODE

if RKsolvable:
    def ode_G(x,Y, coefs=None):
        if SolveMedrano: return Y[0]+3*Y[0]**2
        else: return Y[1]*tf.math.sin(Y[0])-Y[2]*Y[0] # Generic ODE:
                                                      # well solved for order=3,
                                                      # x0 = 0, Y0 = [-1/2,0,1] and x in (-1,2)
                                                      # 2000 epochs and LR e-4

def ode_F(x,Ytilde,coefs=None):
    if RKsolvable:
        return Ytilde[-1]-ode_G(x,Ytilde[:-1],coefs)
    else:
        return Ytilde[-1]*Ytilde[0]+tf.math.tan(x**2) # Generic non-RKsolvable ODE:
                                                      # 'solved' for order=3,
                                                      # x0 = 0, Y0 = [-1/2,0,1] and x in (-1,2)
                                                      # 3000 epochs and LR e-4

# We set the interval of interest and the initial condition

# Initial conditions.
    # If some are not necessary (Order<3) provide arbitrary values, they will be ignored
x0 = 0.0
y0 = -0.5
dy_dx0 = 0.0
d2y_dx20 = 1.0

Y0 = np.array([y0,dy_dx0,d2y_dx20])
Y0 = Y0[:Order]

# Interval of interest
xmin = -1.0
xmax = 2.0

## Computation of the solution by the RK4 method

In [ ]:
if RKsolvable:
    def ode_H(x,Y,coefs=None):
        H=Y.copy()
        H=np.delete(H,0)
        return np.append(H,ode_G(x,Y,coefs))

In [ ]:
if RKsolvable:
    from scipy.integrate import odeint

    xminRK = x0
    xmaxRK = xmax
    npointsRK = 101
    x_RK=np.linspace(xminRK,xmaxRK,npointsRK)

    Y_RK = odeint(ode_H, Y0, x_RK, tfirst=True)

## Definition of the PINN

In [ ]:
class ODE_upto3rd(tf.keras.Model):
    loaded = 0

    def load_ivp(self,F,x0,Y0): # load the Initial Value Problem
        if not type(F) is types.FunctionType: raise ValueError("The provided function is not a function.")
        if not type(Y0) is np.ndarray: raise ValueError("Y0 must be an array.")
        self.function = F
        self.init_cond = (x0,Y0)
        self.ivp_order = len(Y0)
        self.loaded = 1

    def train_step(self, data):
        '''
        Training ocurrs here
        '''
        if not self.loaded:
            raise ValueError("The model hasn't been assigned any initial value problem.")

        x = data

        #Set initial conditions for the PINN
        x0_NN=tf.constant([self.init_cond[0]], dtype=tf.float32)
        y0_exact=tf.constant([self.init_cond[1][0]], dtype=tf.float32)
        dy_dx0_exact=tf.constant([self.init_cond[1][1]], dtype=tf.float32)
        d2y_dx20_exact=tf.constant([self.init_cond[1][2]], dtype=tf.float32)

        with tf.GradientTape() as tape:
            tape.watch(x)
            tape.watch(x0_NN)
            # Calculate the gradients dy3/dx3, dy2/dx2, dy/dx
            with tf.GradientTape() as tape01:
                    tape01.watch(x0_NN)
                    with tf.GradientTape() as tape02:
                        tape02.watch(x0_NN)
                        y0_NN = self(x0_NN,training=False)
                    dy_dx0_NN = tape02.gradient(y0_NN, x0_NN)
            d2y_dx20_NN = tape01.gradient(dy_dx0_NN, x0_NN)
            with tf.GradientTape() as tape1:
                tape1.watch(x)
                with tf.GradientTape() as tape2:
                    tape2.watch(x)
                    with tf.GradientTape() as tape3:
                        tape3.watch(x)
                        y_NN = self(x,training=False)
                    dy_dx_NN = tape3.gradient(y_NN, x)
                d2y_dx2_NN = tape2.gradient(dy_dx_NN, x)
            d3y_dx3_NN = tape1.gradient(d2y_dx2_NN,x)

            #Loss= ODE+ boundary/initial conditions
            y0_exact=tf.reshape(y0_exact,shape=y_NN[0].shape)
            dy_dx0_exact=tf.reshape(dy_dx0_exact,shape=dy_dx0_NN.shape)
            d2y_dx20_exact=tf.reshape(d2y_dx20_exact,shape=d2y_dx20_NN.shape)

            y_NN = tf.reshape(y_NN,dy_dx_NN.shape)
            Ytilde_NN = tf.TensorArray(tf.float32, size=0, dynamic_size=True, clear_after_read=False)
            Ytilde_NN = Ytilde_NN.write(0,y_NN)
            Ytilde_NN = Ytilde_NN.write(1,dy_dx_NN)
            if self.ivp_order >= 2:
                Ytilde_NN = Ytilde_NN.write(2,d2y_dx2_NN)
            if self.ivp_order >= 3:
                Ytilde_NN = Ytilde_NN.write(3,d3y_dx3_NN)
            Ytilde_NN = Ytilde_NN.stack()

            func_shape=self.function(x,Ytilde_NN).shape
            zeros=tf.zeros(func_shape, dtype=tf.float32)

            loss = self.compiled_loss(y0_NN,y0_exact)\
                  +self.compiled_loss(zeros,self.function(x,Ytilde_NN))
            if self.ivp_order >= 2:
                  loss = loss+self.compiled_loss(dy_dx0_NN,dy_dx0_exact)
            if self.ivp_order >= 3:
                loss = loss+self.compiled_loss(d2y_dx20_NN,d2y_dx20_exact)

        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        return {m.name: m.result() for m in self.metrics}


## Run the PINN

In [ ]:
n_train = 11

# Definition of the function domain
x_train=np.linspace(xmin,xmax,n_train)

# Add initial conditions to the training set
np.append(x_train,x0)

# Input and output neurons (from the data)
input_neurons  = 1
output_neurons = 1

# Hiperparameters
epochs = 2000

# Define the model
activation='tanh'
input=Input(shape=(input_neurons,))
x=Dense(50, activation=activation)(input)
x=Dense(50, activation=activation)(x)
x=Dense(50, activation=activation)(x)
output = Dense(output_neurons,activation=None)(x)
model = ODE_upto3rd(input,output)
model.load_ivp(ode_F,x0,Y0)

#Define the metrics, optimizer and loss
loss = tf.keras.losses.MeanSquaredError()
metrics = tf.keras.metrics.MeanSquaredError()
optimizer = Adam(learning_rate=0.0001)

model.compile(loss=loss,
          optimizer=optimizer,
          metrics=[metrics])
model.summary()

# Just use `fit` as usual
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=1000,
                                            restore_best_weights=True)

history = model.fit(x_train, batch_size=1, epochs=epochs,
                  callbacks=callback)

## Check the PINN

### Plot losses and metrics

In [ ]:
# summarize history for loss and metris
plt.rcParams['figure.dpi'] = 150
plt.plot(history.history['loss'],color='magenta',linewidth=0.5,label='Total losses ($L_D + L_B$)')
#plt.plot(history.history['mean_squared_error'],color='cyan',linewidth=0.5,label='MSE')
plt.yscale("log")
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show()

### Visual check of the results

In [ ]:
# Check the PINN at different points not included in the training set
n = 500
x = np.linspace(xmin-1,xmax+1,n)
x0 = 0.0
y_NN = model.predict(x)

# We compute the derivatives and build Ytilde_NN again
x_tf = tf.convert_to_tensor(x, dtype=tf.float32)

with tf.GradientTape() as t1:  # persistent=True doesn't appear to be necessary
    t1.watch(x_tf)
    with tf.GradientTape() as t2:
        t2.watch(x_tf)
        with tf.GradientTape() as t3:
            t3.watch(x_tf)
            y_NN = model(x_tf)
        dy_dx_NN = t3.gradient(y_NN, x_tf)
    d2y_dx2_NN = t2.gradient(dy_dx_NN, x_tf)
d3y_dx3_NN = t1.gradient(d2y_dx2_NN, x_tf)

y_NN = tf.reshape(y_NN,dy_dx_NN.shape)
Ytilde_NN = tf.TensorArray(tf.float32, size=0, dynamic_size=True, clear_after_read=False)
Ytilde_NN = Ytilde_NN.write(0,y_NN)
Ytilde_NN = Ytilde_NN.write(1,dy_dx_NN)
if Order >= 2:
    Ytilde_NN = Ytilde_NN.write(2,d2y_dx2_NN)
if Order >= 3:
    Ytilde_NN = Ytilde_NN.write(3,d3y_dx3_NN)
Ytilde_NN = Ytilde_NN.stack()

# Define Graph parameters
colours = ("darkred","firebrick","coral","rosybrown")
linestyles = ("dashed","dotted","dotted","dotted")
labels = ("$y_{NN}(x)$","$y'_{NN}(x)$","$y''_{NN}(x)$","$y'''_{NN}(x)$")

# Plot the ODE solution
plt.rcParams['figure.dpi'] = 150
if RKsolvable:
    plt.plot(x_RK, Y_RK[:,0], color="turquoise", label="$y_{RK}(x)$")
for i in range(Order+1):
    plt.plot(x, Ytilde_NN[i], color=colours[i], linestyle=linestyles[i], linewidth=1, label=labels[i])
plt.legend(bbox_to_anchor=(1, 1), loc="upper left")
plt.title("Solution of the ODE")
plt.xlabel("x")
plt.show()

# Check the ODE resolution
if RKsolvable:
    plt.plot(x, ode_G(x,Ytilde_NN[:-1]), color="plum", linewidth=1, label="$G(x,Y)$")
    plt.plot(x, Ytilde_NN[Order], color=colours[Order], linestyle=linestyles[Order], linewidth=1, label=labels[Order])
plt.plot(x, ode_F(x_tf,Ytilde_NN), color="darkblue", linewidth=1, label="$F(x,Y_{\t{tilde}})$")
plt.plot(xmin,ode_F(xmin,Ytilde_NN[:,np.where((x>=xmin))[0][0]]),'b+',label="$(x_{\t{min}},x_{\t{max}})$")
plt.plot(xmax,ode_F(xmax,Ytilde_NN[:,np.where((x<=xmax))[0][-1]]),'b+')
plt.legend(bbox_to_anchor=(1, 1), loc="upper left")
plt.title("Result comparison")
plt.xlabel("x")
plt.show()

In the non-RKsolvable example the solution seems to jump to another one at around 1.2 . There is no other way to check the goodness of the solution but comparing $F$ to 0.